In [1]:
#from textblob import TextBlob
#from attributegetter import *
from generatengrams import ngrammatch
from Contexts import *
import json
from Intents import *
from spellcheck import *
import random
import os
import re
from termcolor import colored

In [2]:
def listMovies(attributes):
    return ""

def listRestarunts(attributes):
    return ""

In [3]:
def check_actions(current_intent, attributes, context):
    '''This function performs the action for the intent
    as mentioned in the intent config file'''
    '''Performs actions pertaining to current intent
    for action in current_intent.actions:
        if action.contexts_satisfied(active_contexts):
            return perform_action()
    '''    
    if current_intent.name == "BookMovie":
        listMovies(attributes)
    elif current_intent.name == "RestaruntSearch":
        listRestarunts(attributes)
    
    context = IntentComplete()
    return 'action: ' + current_intent.action, context

def check_required_params(current_intent, attributes, context):
    '''Collects attributes pertaining to the current intent'''
    
    for para in current_intent.params:
         if para.required:
            if para.name not in attributes:
#                print("parametres       ",para.name)
#                print("atrributes         ",attributes)
#                print("current_intent         ",current_intent.name)
#                print("context         ",para.context)
                if para.name == 'Getconfirmation':
                    print("Please confirm if we can book the restaurant for ", attributes)
                    context = Getconfirmation()                        
                if (current_intent.name == 'SearchMovie') and attributes != {}  :
                    print("Do yo want to search with the entered attributes ",attributes)                   
                    
                return random.choice(para.prompts), context
    return None, context


def input_processor(user_input, context, attributes, intent):
    '''Spellcheck and entity extraction functions go here'''
    
    #uinput = TextBlob(user_input).correct().string
    
    #update the attributes, abstract over the entities in user input
    attributes, cleaned_input = getattributes(user_input, context, attributes,intent)
    
    return attributes, cleaned_input

def loadIntent(path, intent):
    with open(path) as fil:
        dat = json.load(fil)
        intent = dat[intent]
        return Intent(intent['intentname'],intent['intentenglishname'],intent['Parameters'],intent['actions'])

def intentIdentifier(clean_input, context,current_intent):
    clean_input = clean_input.lower()
    scores = ngrammatch(clean_input)
    scores = sorted_by_second = sorted(scores, key=lambda tup: tup[1])
    # print clean_input
    #print 'scores', scores
    

    if(current_intent==None):
        if(clean_input=="search"):
            return loadIntent('params/MovieParams2.cfg', 'SearchMovie')
        if(clean_input=='book'):
            return loadIntent('params/MovieParams2.cfg','BookRestaurant')
        else:
            return loadIntent('params/MovieParams2.cfg',scores[-1][0])
    else:
        #print 'same intent'
        return current_intent

def getattributes(uinput,context,attributes,intent):
    '''This function marks the entities in user input, and updates
    the attributes dictionary'''
    #Can use context to to context specific attribute fetching
    if context.name.startswith('IntentComplete'):
        return attributes, uinput
    else:

        files = os.listdir('./entities/')
        entities = {}
        for fil in files:
            lines = open('./entities/'+fil).readlines()            
            for i, line in enumerate(lines):
                lines[i] = line[:-1]
                #print(lines[i])
            entities[fil[:-4]] = '|'.join(lines)
            #print(entities[fil[:-4]])
        #print(entities)

        for entity in entities:
            for i in entities[entity].split('|'):
                if i.lower() in uinput.lower():
                    attributes[entity] = i
        for entity in entities:
                uinput = re.sub(entities[entity],r'$'+entity,uinput,flags=re.IGNORECASE)
        
        if context.name =='Getconfirmation'   and context.active: 
            if uinput.lower() == 'yes':
                #print(uinput.lower())
                attributes['Getconfirmation'] = uinput.lower()
                #print(attributes)
                
                context.active = False
            if uinput.lower() == 'no':
                print("Please enter the details for the  ", intent.englishname )
                attributes ={}
                context.active = True
            
        
    return attributes, uinput

In [6]:
class Session:
    def __init__(self, attributes=None, active_contexts=[FirstGreeting(), IntentComplete() ]):
        
        '''Initialise a default session'''
        
        #Contexts are flags which control dialogue flow, see Contexts.py
        self.active_contexts = active_contexts
        self.context = FirstGreeting()
        
        #Intent tracks the current state of dialogue
        #self.current_intent = First_Greeting()
        self.current_intent = None
        
        #attributes hold the information collected over the conversation
        self.attributes = {}
        
    def update_contexts(self):
        '''Not used yet, but is intended to maintain active contexts'''
        for context in self.active_contexts:
            if context.active:
                context.decrease_lifespan()

    def reply(self, user_input):
        '''Generate response to user input'''

        user_input = split_line(user_input)
        print("corrected: ",user_input)
        self.attributes, clean_input = input_processor(user_input, self.context, self.attributes, self.current_intent)
        
        self.current_intent = intentIdentifier(clean_input, self.context, self.current_intent)
        
        prompt, self.context = check_required_params(self.current_intent, self.attributes, self.context)

        #prompt being None means all parameters satisfied, perform the intent action
        if prompt is None:
            if self.context.name!='IntentComplete':
                prompt, self.context = check_actions(self.current_intent, self.attributes, self.context)
        
        #Resets the state after the Intent is complete
        if self.context.name=='IntentComplete':
            self.attributes = {}
            self.context = FirstGreeting()
            self.current_intent = None
        
        return prompt

In [ ]:
session = Session()

print (colored('\033[1m' + 'BOT:','red'), colored('\033[1m'+'Hi! How may I assist you?','green'))

while True:
    inp = input('User: ')
    if inp == 'exit':
        print(colored('\033[1m' + 'Bye then!','red'))
        break
    else:   
        print(colored('\033[1m' + 'BOT:','red'), colored('\033[1m' + session.reply(inp),'green'))

BOT: Hi! How may I assist you?
User: movieee
corrected:  movie
BOT: Please specify movie language(Hindi,Telugu,English)?


In [ ]:
attributes = {}
context = FirstGreeting()
current_intent = None
user_input = 'low'

attributes, clean_input = input_processor(user_input, context, attributes, current_intent)
print(attributes,clean_input)

In [8]:
getattributes(user_input,context,attributes)

Dwayne Johnson|Rani Mukherji|Nikhil|Prabhas|Ajay Devgan
Low|Medium|Hig
Chinese|Italian|Mexican|Indian
Today|Tomorrow|Dayafte
Action|Comedy|Famil
Telugu|English|Hind
North|East|South|Wes
{'Actor': 'Dwayne Johnson|Rani Mukherji|Nikhil|Prabhas|Ajay Devgan', 'costtype': 'Low|Medium|Hig', 'cuisine': 'Chinese|Italian|Mexican|Indian', 'date': 'Today|Tomorrow|Dayafte', 'genre': 'Action|Comedy|Famil', 'language': 'Telugu|English|Hind', 'location': 'North|East|South|Wes'}


({'costtype': 'Low'}, '$costtype')

In [10]:
current_intent = intentIdentifier(clean_input,context, current_intent)

In [11]:
current_intent.name

'BookRestaurant'